In [54]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from itertools import chain

In [55]:
data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/modelv2'

In [56]:
word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
word_state_size = 100
other_state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 10

In [57]:
with tf.name_scope("input"):
    path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length")
    word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids")
    pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids")
    dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids")
    y = tf.placeholder(tf.int32, [batch_size], name="y")

In [58]:
with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embedded_word = tf.nn.embedding_lookup(W, word_ids)
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

In [59]:
with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

In [60]:
with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [61]:
word_hidden_state = tf.zeros([batch_size, word_state_size], name='word_hidden_state')
word_cell_state = tf.zeros([batch_size, word_state_size], name='word_cell_state')
word_init_state = tf.contrib.rnn.LSTMStateTuple(word_hidden_state, word_cell_state)

In [62]:
other_hidden_states = tf.zeros([channels-1, batch_size, other_state_size], name="hidden_state")
other_cell_states = tf.zeros([channels-1, batch_size, other_state_size], name="cell_state")

In [63]:
other_init_states = [tf.contrib.rnn.LSTMStateTuple(other_hidden_states[i], other_cell_states[i]) for i in range(channels-1)]

In [11]:
with tf.variable_scope("word_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=word_init_state)
    state_series_word1 = tf.reduce_max(state_series, axis=1)

In [12]:
with tf.variable_scope("word_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=word_init_state)
    state_series_word2 = tf.reduce_max(state_series, axis=1)

In [13]:
with tf.variable_scope("pos_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=other_init_states[0])
    state_series_pos1 = tf.reduce_max(state_series, axis=1)

In [14]:
with tf.variable_scope("pos_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=other_init_states[0])
    state_series_pos2 = tf.reduce_max(state_series, axis=1)

In [15]:
with tf.variable_scope("dep_lstm1"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=other_init_states[1])
    state_series_dep1 = tf.reduce_max(state_series, axis=1)

In [16]:
with tf.variable_scope("dep_lstm2"):
    cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
    state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=other_init_states[1])
    state_series_dep2 = tf.reduce_max(state_series, axis=1)

In [64]:
state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

In [65]:
state_series = tf.concat([state_series1, state_series2], 1)

In [66]:
state_series

<tf.Tensor 'concat_5:0' shape=(10, 600) dtype=float32>

In [67]:
with tf.name_scope("hidden_layer"):
    W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = tf.matmul(state_series, W) + b

In [21]:
with tf.name_scope("dropout"):
    h_drop = tf.nn.dropout(y_hidden_layer, 0.3)

In [22]:
with tf.name_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_hidden_layer, W) + b
    predictions = tf.argmax(logits, 1)

In [23]:
global_step = tf.Variable(0, name="global_step")

In [24]:
tv_all = tf.trainable_variables()
tv_regu = []
non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('biases')==-1):
            tv_regu.append(t)


In [25]:
with tf.name_scope("loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    total_loss = loss + l2_loss

In [26]:
with tf.name_scope("accuracy"):
    correct_predictions = tf.equal(predictions, tf.arg_max(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy") 

In [27]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

In [28]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

In [29]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

In [30]:
word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

In [31]:
unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

In [32]:
f = open('data/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

In [33]:
relations = []
for line in open('data/train_relation.txt'):
    relations.append(line.strip().split()[1])

In [34]:
num_batches = int(8000/batch_size)
num_batches

800

In [35]:
pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

In [36]:
dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

In [37]:
relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

In [38]:
rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

In [39]:
pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

In [40]:
dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

In [41]:
pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

In [42]:
dep2id['OTH'] = 20
id2dep[20] = 'OTH'

In [43]:
JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

In [44]:
 def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [45]:
dep = []
for i in range(8000):
    dep.append(dep_p1[i])

In [46]:
for i in range(8000):
    dep.append(dep_p2[i])

In [47]:
dep_freq = nltk.FreqDist(chain(*dep))
dep_types = dep_freq.most_common(20)

In [48]:
for i in range(8000):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

In [49]:
word_p1_ids = np.ones([8000, max_len_path],dtype=int)
word_p2_ids = np.ones([8000, max_len_path],dtype=int)
pos_p1_ids = np.ones([8000, max_len_path],dtype=int)
pos_p2_ids = np.ones([8000, max_len_path],dtype=int)
dep_p1_ids = np.ones([8000, max_len_path],dtype=int)
dep_p2_ids = np.ones([8000, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

In [50]:
for i in range(8000):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [52]:
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [53]:
model = tf.train.latest_checkpoint(model_dir)
saver.restore(sess, model)

INFO:tensorflow:Restoring parameters from checkpoint/modelv2/model


NotFoundError: Key word_embedding/W/Adam_1 not found in checkpoint
	 [[Node: save/RestoreV2_47 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_47/tensor_names, save/RestoreV2_47/shape_and_slices)]]

Caused by op 'save/RestoreV2_47', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shanu/virtualenv/tensorflow3/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shanu/virtualenv/tensorflow3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shanu/virtualenv/tensorflow3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/shanu/virtualenv/tensorflow3/lib/python3.

In [53]:
# latest_embd = tf.train.latest_checkpoint(word_embd_dir)
# word_embedding_saver.restore(sess, latest_embd)

In [57]:
num_epochs = 10
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step % 10 ==0:
            print("Step:", step, "loss:",loss)
        if step % 1000 == 0:
            saver.save(sess, model_dir + '/model', global_step = step)
            print("Saved Model")

Step: 10 loss: 2.80779
Step: 20 loss: 2.91857
Step: 30 loss: 2.5435
Step: 40 loss: 2.77815
Step: 50 loss: 2.86619
Step: 60 loss: 2.70983
Step: 70 loss: 2.59677
Step: 80 loss: 2.57367
Step: 90 loss: 2.63232
Step: 100 loss: 2.67632
Step: 110 loss: 2.46945
Step: 120 loss: 2.41762
Step: 130 loss: 2.47456
Step: 140 loss: 2.18848
Step: 150 loss: 2.51237
Step: 160 loss: 2.62843
Step: 170 loss: 2.14578
Step: 180 loss: 2.19004
Step: 190 loss: 2.36966
Step: 200 loss: 2.43195
Step: 210 loss: 2.35718
Step: 220 loss: 2.12848
Step: 230 loss: 2.46286
Step: 240 loss: 1.78023
Step: 250 loss: 1.88158
Step: 260 loss: 2.60348
Step: 270 loss: 2.12932
Step: 280 loss: 1.77932
Step: 290 loss: 2.52527
Step: 300 loss: 2.03072
Step: 310 loss: 1.65725
Step: 320 loss: 2.38889
Step: 330 loss: 1.7463
Step: 340 loss: 2.76068
Step: 350 loss: 3.97415
Step: 360 loss: 1.99721
Step: 370 loss: 1.42537
Step: 380 loss: 1.83674
Step: 390 loss: 2.24145
Step: 400 loss: 2.22669
Step: 410 loss: 2.5695
Step: 420 loss: 1.93245
Step

Step: 3270 loss: 0.873815
Step: 3280 loss: 0.192159
Step: 3290 loss: 0.425214
Step: 3300 loss: 0.542901
Step: 3310 loss: 0.241862
Step: 3320 loss: 0.698558
Step: 3330 loss: 0.148543
Step: 3340 loss: 0.203272
Step: 3350 loss: 0.789588
Step: 3360 loss: 1.136
Step: 3370 loss: 0.315635
Step: 3380 loss: 0.449273
Step: 3390 loss: 0.45932
Step: 3400 loss: 0.363575
Step: 3410 loss: 0.183468
Step: 3420 loss: 0.223123
Step: 3430 loss: 0.228775
Step: 3440 loss: 0.37514
Step: 3450 loss: 0.464133
Step: 3460 loss: 0.292304
Step: 3470 loss: 0.339957
Step: 3480 loss: 0.505914
Step: 3490 loss: 0.207361
Step: 3500 loss: 0.17801
Step: 3510 loss: 0.313443
Step: 3520 loss: 0.446755
Step: 3530 loss: 0.290929
Step: 3540 loss: 0.615026
Step: 3550 loss: 0.530993
Step: 3560 loss: 0.29253
Step: 3570 loss: 0.279436
Step: 3580 loss: 0.237378
Step: 3590 loss: 0.886074
Step: 3600 loss: 0.468314
Step: 3610 loss: 0.237626
Step: 3620 loss: 0.482566
Step: 3630 loss: 0.191543
Step: 3640 loss: 0.158047
Step: 3650 loss: 0.

Step: 6430 loss: 0.275526
Step: 6440 loss: 0.180767
Step: 6450 loss: 0.146435
Step: 6460 loss: 0.13959
Step: 6470 loss: 0.319911
Step: 6480 loss: 0.138233
Step: 6490 loss: 0.242108
Step: 6500 loss: 0.208529
Step: 6510 loss: 0.446008
Step: 6520 loss: 0.146951
Step: 6530 loss: 0.149112
Step: 6540 loss: 0.196982
Step: 6550 loss: 0.507518
Step: 6560 loss: 0.751069
Step: 6570 loss: 0.255289
Step: 6580 loss: 0.548136
Step: 6590 loss: 0.149067
Step: 6600 loss: 0.248962
Step: 6610 loss: 0.137674
Step: 6620 loss: 0.153976
Step: 6630 loss: 0.26702
Step: 6640 loss: 0.137183
Step: 6650 loss: 0.417848
Step: 6660 loss: 0.248134
Step: 6670 loss: 0.145402
Step: 6680 loss: 0.157638
Step: 6690 loss: 0.177348
Step: 6700 loss: 0.888012
Step: 6710 loss: 0.146005
Step: 6720 loss: 0.14551
Step: 6730 loss: 0.179755
Step: 6740 loss: 0.142292
Step: 6750 loss: 0.14972
Step: 6760 loss: 0.135946
Step: 6770 loss: 0.15104
Step: 6780 loss: 0.148572
Step: 6790 loss: 0.251365
Step: 6800 loss: 0.233396
Step: 6810 loss: 

In [58]:
print("Training Complete")

Training Complete


In [59]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)
count = 0
for i in range(800):
    count += sum(all_predictions[i] == rel_ids[i*10:(i+1)* 10])
print("accuracy", count/8000* 100)

accuracy 94.175
